In [1]:
# Notebook cell 1: Imports
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# Notebook cell 2: Load CSV and Check
df = pd.read_csv('data/sample_sales.csv')
df['ds'] = pd.to_datetime(df['ds'])
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df = df.dropna().sort_values('ds').reset_index(drop=True)
display(df.head())
print("Rows:", len(df))

# Notebook cell 3: Prophet Forecast & Components
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(periods=0)
forecast = m.predict(future)

plt.figure(figsize=(7,4))
plt.plot(df['ds'], df['y'], label='Actual')
plt.plot(forecast['ds'], forecast['yhat'], label='Forecast')
plt.legend()
plt.title('Prophet Forecast')
plt.show()

# Prophet Components Plot (trend/seasonality)
try:
    fig = m.plot_components(forecast)
    plt.show()
except Exception as e:
    print(f"Could not render components: {e}")

# Notebook cell 4: LSTM Model (Basic)
n_lags = 12
series = df['y'].values

# Prepare data
scaler = MinMaxScaler()
series_scaled = scaler.fit_transform(series.reshape(-1,1)).flatten()
X, y_lstm = [], []
for i in range(n_lags, len(series_scaled)):
    X.append(series_scaled[i-n_lags:i])
    y_lstm.append(series_scaled[i])
X = np.array(X)[..., None]
y_lstm = np.array(y_lstm)

# Build & train LSTM
model = Sequential([
    LSTM(64, input_shape=(n_lags,1)),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y_lstm, epochs=100, batch_size=8, verbose=0)

# Forecast future points
preds_scaled = []
last_vals = series_scaled[-n_lags:]
for _ in range(12):  # Next 12 months for example
    inpt = last_vals[-n_lags:].reshape(1, n_lags, 1)
    pred = model.predict(inpt)[0,0]
    preds_scaled.append(pred)
    last_vals = np.append(last_vals, pred)
preds = scaler.inverse_transform(np.array(preds_scaled).reshape(-1,1)).flatten()

future_idx = pd.date_range(df['ds'].max(), periods=13, freq='MS')[1:]
plt.figure(figsize=(7,4))
plt.plot(df['ds'], df['y'], label='Actual')
plt.plot(future_idx, preds, label='LSTM Forecast')
plt.legend()
plt.title('LSTM Forecast')
plt.show()

# Notebook cell 5: Evaluation Example
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(df['y'][-12:], forecast['yhat'][-12:])
rmse = mean_squared_error(df['y'][-12:], forecast['yhat'][-12:], squared=False)
print(f"Prophet MAE: {mae:.2f} | RMSE: {rmse:.2f}")

c:\Users\acer\OneDrive\Desktop\FUTURE_ML_01\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'data/sample_sales.csv'